In [69]:
import httpx
from rdflib import Graph
from pysolr import Solr
from api.src.function.bibframe.Work.graphWork import MakeGraphWork

In [39]:
from typing import Optional
from pydantic import BaseModel
from api.src.schemas.metadata.bibframe.adminMetadata import AdminMetadata
from api.src.schemas.metadata.bibframe.element import Element
from api.src.schemas.metadata.bibframe.title import Title
from api.src.schemas.metadata.bibframe.classification import Classification
from api.src.schemas.metadata.bibframe.contribution import Contribution


class Work(BaseModel):
    adminMetadata: AdminMetadata
    type: list[str]
    content: list[Element]
    language: list[Element]
    title: Title
    classification: Optional[Classification]
    contribution: Optional[list[Contribution]]
    subject: Optional[list[Element]]   
    genreForm: Optional[list[Element]]
    note: Optional[str]
    summary: Optional[str]
    tableOfContents: Optional[str]
    supplementaryContent: Optional[list[Element]]
    illustrativeContent: Optional[list[Element]]
    intendedAudience: Optional[list[Element]]
    geographicCoverage: Optional[list[Element]]
    

In [10]:
uri = 'http://id.loc.gov/resources/works/1893399'

Subject

In [14]:
response = httpx.get('http://localhost:8000/import/loc/subjects?uri=http%3A%2F%2Fid.loc.gov%2Fauthorities%2Fsubjects%2Fsh99001902')
response.json()

{'type': 'Topic',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '2023-06-14',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-06-14T11:14:28',
  'identifiedBy': [{'type': 'Lccn',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'sh99001902'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'elementList': [{'type': 'TopicElement',
   'elementValue': {'value': 'Metodologia', 'lang': 'pt'}}],
 'note': 'Use como uma subdivisão tópica em disciplinas para trabalhos tanto na teoria quanto na prática dos procedimentos a serem seguidos.',
 'hasReciprocalAuthority': None,
 'hasReciprocalExternalAuthority': None,
 'hasBroaderAuthority': None,
 'hasBroaderExternalAuthority': None,
 'hasNarrowerAuthority': None,
 'hasNarrowerExternalAuthori

In [ ]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/authorities/subject/', headers=headers, json=response.json())
response.json()

In [97]:
graph = Graph()
graph.parse("http://id.loc.gov/resources/instances/1893399.rdf")
graph.serialize('instance.ttl')

<Graph identifier=Nf3e3fc8d151a48df82027e3b0eaa5a69 (<class 'rdflib.graph.Graph'>)>

WORK

In [76]:
uri = 'http://id.loc.gov/resources/works/1893399'
url = f'http://localhost:8000/import/loc/works?uri={uri}'
response = httpx.get(url)
response

<Response [200 OK]>

In [77]:
response.json()

{'adminMetadata': {'encodingLevel': 'menclvl:f',
  'assigner': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '2023-06-14',
  'descriptionConventions': 'http://id.loc.gov/vocabulary/descriptionConventions/isbd',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': {'label': 'BiblioKeia',
   'generationDate': '2023-06-14T15:49:06'},
  'identifiedBy': [{'type': 'Local',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': '1893399'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'type': ['Work', 'Text'],
 'content': [{'label': 'text',
   'lang': None,
   'uri': 'http://id.loc.gov/vocabulary/contentTypes/txt',
   'type': ['http://id.loc.gov/ontologies/bibframe/Content']}],
 'language': [{'label': 'English',
   'lang': 'en',
   'uri': 'http://id.loc.gov/vocabulary/languages/eng',
   'type': ['http://id.loc.g

In [78]:
obj = response.json()

In [79]:
s = [
    {'label': 'Metodologia',
  'lang': 'pt',
  'uri': 'https://bibliokeia.com/authorities/Topic/bkau-1',
  'type': ['http://id.loc.gov/ontologies/bibframe/Topic',
   'http://www.loc.gov/mads/rdf/v1#Topic'] }
    
]
obj['subject'] = s

In [93]:
c = [{'type': ['http://id.loc.gov/ontologies/bflc/PrimaryContribution'],
  'agent': 'https://bibliokeia.com/authorities/PersonalName/bkau-2',
  'label': 'Popper, Karl R. (Karl Raimund), 1902-1994',
  'role': 'http://id.loc.gov/vocabulary/relators/ctb'}]
obj['contribution'] = c

In [95]:
obj['subject']

[{'label': 'Metodologia',
  'lang': 'pt',
  'uri': 'https://bibliokeia.com/authorities/Topic/bkau-1',
  'type': ['http://id.loc.gov/ontologies/bibframe/Topic',
   'http://www.loc.gov/mads/rdf/v1#Topic']}]

In [91]:
request = Work(**obj)
request.contribution

[Contribution(type=['http://id.loc.gov/ontologies/bflc/PrimaryContribution'], agent='http://id.loc.gov/rwo/agents/n80032184', label='Popper, Karl R. (Karl Raimund), 1902-1994', role='http://id.loc.gov/vocabulary/relators/ctb')]

In [96]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/cataloguing/work', headers=headers, json=obj)
response

ReadTimeout: timed out

In [83]:
request.subject

[Element(label='Metodologia', lang='pt', uri='https://bibliokeia.com/authorities/Topic/bkau-1', type=['http://id.loc.gov/ontologies/bibframe/Topic', 'http://www.loc.gov/mads/rdf/v1#Topic'])]

In [72]:
id = 'bkc-1'
SDocs = list()
for subject in request.subject:
    sID = subject.uri.split('/')[-1]
    SDoc = { 
        "id": f"{sID}/subjectOf/{id}",
        "type": [i for i in subject.type],
        "lang": subject.lang,
        "label": subject.label
        }
    doc = {
        "id": sID,
        "subjectOf": { "add" : SDoc }}    
    
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
    
responseSolr =  solr.add([doc], commit=True)
responseSolr


'{\n  "responseHeader":{\n    "status":0,\n    "QTime":170}}\n'

In [88]:
for contribution in request.contribution:
    print(contribution.agent)

http://id.loc.gov/rwo/agents/n80032184


In [48]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/cataloguing/work', headers=headers, json=obj)
response

<Response [201 Created]>

In [ ]:
G = MakeGraphWork(request, 'TESTE')
print(G)

In [26]:
def DocWork(request, id):

    doc = {
        "id": id,
        "type": request.type,
        "content": [i.label for i in request.content],
        "mainTitle": request.title.mainTitle,
        'language': [i.label for i in request.language],
        "subtitle": request.title.subtitle,
        "cdd": request.classification,
        "note": request.note,
        "summary": request.summary,
        "tableOfContents": request.tableOfContents,
        "supplementaryContent": [i.label for i in request.supplementaryContent] if request.supplementaryContent else None,
        "illustrativeContent": [i.label for i in request.illustrativeContent] if request.illustrativeContent else None,
        "intendedAudience": [i.label for i in request.intendedAudience] if request.intendedAudience else None,
        }
    
    if request.contribution:
        contributions = list()
        for i in request.contribution:
            c = { "id": f"{id}/contribution/{i.agent.split('/')[-1]}",
                "type": [i.split('/')[-1] for i in i.type],
                "agent": i.agent,
                "label": i.label,
                "role": i.role }
            contributions.append(c)
        doc['contribution'] = contributions   

    if request.subject:
        pass
    if request.genreForm:
        pass
    return doc 

    # subjects = list()
    # for subject in request.subjects:
    #     subjects.append(subject.label)
    # doc['subjects'] = subjects

    # solr.add([doc], commit=True)

doc = DocWork(request, "bk-1")
doc


{'id': 'bk-1',
 'type': ['Work', 'Text'],
 'content': ['text'],
 'mainTitle': 'A Floresta Amazônica garante nossas vidas',
 'language': ['Portuguese'],
 'subtitle': None,
 'cdd': None,
 'note': None,
 'summary': None,
 'tableOfContents': None,
 'supplementaryContent': ['bibliography'],
 'illustrativeContent': None,
 'intendedAudience': None,
 'contribution': [{'id': 'bk-1/contribution/n2022250451',
   'type': ['PrimaryContribution'],
   'agent': 'http://id.loc.gov/rwo/agents/n2022250451',
   'label': 'Csettkey, Marcelo',
   'role': 'http://id.loc.gov/vocabulary/relators/aut'}]}

In [27]:
solr = Solr('http://localhost:8983/solr/acervo/', timeout=10)
responseSolr = solr.add([doc], commit=True)